# Deep reinforcement learning assignment - the game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. 

To this end we introduce two main classes:
- Environment: an abstract class that represents the states, rewards, and actions to obtain the new state.
- Agent: interact with the Environment by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$

And we will consider:
- a Random Agent
- an Agent that acts $\epsilon$-greedily (Q-value function approximation by means of DQN-learning algorithm, using a full connected NN)
- an Agent that acts $\epsilon$-greedily (Q-value function approximation by means of DQN-learning algorithm, using a CNN to incorporate spatial information)
- an Agent that has disincentives to rivisit previous states and that acts $\epsilon$-greedily $\epsilon$ with $\epsilon$ that changes dynamically

# Importing librarires

In [ ]:
!pip install scikit-video
!pip install keras
!pip install opencv-python

     |████████████████████████████████| 2.3MB 5.4MB/s 


In [ ]:
import keras
import numpy as np
import scipy
import io
import base64
from IPython.display import HTML
import skvideo
import skvideo.io
import cv2
import json
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from tensorflow.python.keras.optimizers import *
from keras.layers import Conv2D,MaxPooling2D,Activation,AveragePooling2D,Reshape,BatchNormalization,Dropout,Flatten, Convolution2D
from keras.utils import np_utils
import tensorflow as tf

# Defining abstract classes: Environment and Agent

***
### The Environment

In [ ]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        """
        board is a matrix whose elements can be 0, +0.5, -1 representing rispectively the presence of nothing, cheese and poisoned cheese.
        position is a matrix that contains a one in the current position of the rat.
        """
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board<0,2] = 256
        b[self.x,self.y,:]= 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over, reward

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

# The Agent

In [ ]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

# Hyperparameters and utility methods of the Game

In [ ]:
# parameters
size = 13
T=50
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

## A1) Random Agent

In [ ]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, 4, size=1)[0]

In [ ]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over, reward_1 = env.act(action, False)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        ##### FILL IN HERE
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [ ]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 3.5/2.0. Average score (1.5)
Win/lose count 3.0/3.0. Average score (0.75)
Win/lose count 3.0/2.0. Average score (0.8333333333333334)
Win/lose count 1.0/8.0. Average score (-1.125)
Win/lose count 3.5/6.0. Average score (-1.4)
Win/lose count 4.5/2.0. Average score (-0.75)
Win/lose count 4.5/3.0. Average score (-0.42857142857142855)
Win/lose count 3.0/6.0. Average score (-0.75)
Win/lose count 3.5/5.0. Average score (-0.8333333333333334)
Win/lose count 4.5/4.0. Average score (-0.7)
Final score: -0.7


***
## A2) DQN - FC

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

In [ ]:
class Memory(object):
     """
     The Memory class stores moves (in a replay buffer) 
     via remember and provides a random_access to these.
     """
     def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

     def remember(self, m):
        if(len(self.memory) < self.max_memory):
            self.memory.append(m)
            self.max_memory += 1
        else:
            self.memory[np.random.randint(len(self.memory))] = m

     def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

In [ ]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            
            action = agent.act(state)
                        
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over, reward_1 = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [ ]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size,5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            
            episode = self.memory.random_access()
                        
            if episode[4]:
                ######## FILL IN
                input_states[i] = episode[0]
                target_q[i,:] = self.model.predict(episode[0].reshape((1,5,5,self.n_state)))
                target_q[i,int(episode[2])] = episode[3]
            else:
                ######## FILL IN
                input_states[i] = episode[0]
                target_q[i,:] = self.model.predict(episode[0].reshape((1,5,5,self.n_state)))
                target_q[i,int(episode[2])] = episode[3] + self.discount*int(np.amax(self.model.predict(episode[1].reshape((1,5,5,self.n_state))))) 

            
        # instead of clipping we use tanh as activation function of the last layer
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        model.run_eagerly = True
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        
        model = keras.Sequential()
        model.add(Dense(250,activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Dense(1000,activation='relu'))
        model.add(Dense(25,activation='relu'))
        model.add(Flatten())
        model.add(Dense(4,activation='tanh'))
        opt = tf.keras.optimizers.SGD(learning_rate=0.1, decay=1e-4, momentum=0.0)
        model.compile(opt, "mse")
        model.run_eagerly = True
        self.model = model
        model.summary()
    
    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))

In [ ]:
T = 50
epochs_train = 50
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.4, memory_size=2000, batch_size = 32, n_state=2)
train(agent, env, epochs_train, prefix='fc_train10')
HTML(display_videos('fc_train100.mp4'))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5, 5, 250)         750       
_________________________________________________________________
dense_1 (Dense)              (None, 5, 5, 1000)        251000    
_________________________________________________________________
dense_2 (Dense)              (None, 5, 5, 25)          25025     
_________________________________________________________________
flatten (Flatten)            (None, 625)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 2504      
Total params: 279,279
Trainable params: 279,279
Non-trainable params: 0
_________________________________________________________________
Epoch 000/050 | Loss 0.0206 | Win/lose count 1.5/6.0 (-4.5)
Epoch 001/050 | Loss 0.0271 | Win/lose count 3.5/6.0 (-2

***
## A3) DQN - CNN

In [ ]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        nb_actions = 4
        model = Sequential()
        model.add(Convolution2D(32, (2, 2), input_shape=(5,5,self.n_state))) 
        model.add(Convolution2D(64, (2, 2))) 
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation('relu'))
        model.add(Dense(nb_actions))
        model.add(Activation('tanh')) 
        opt = tf.keras.optimizers.SGD(learning_rate=0.1, decay=1e-4, momentum=0.0)
        model.compile(opt, "mse")
        self.model = model
        model.summary()
        
    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))

In [ ]:
T = 50
epochs_train = 50

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.4, memory_size=2000, batch_size = 32, n_state=2)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 3, 64)          8256      
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              590848    
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 4100      
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0

# A comparison - DQN FC VS DQN CNN

In [ ]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          8256      
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              590848    
_________________________________________________________________
activation_2 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 4100      
_________________________________________________________________
activation_3 (Activation)    (None, 4)                

FileNotFoundError: ignored

In [ ]:
HTML(display_videos('cnn_test0.mp4'))

In [ ]:
HTML(display_videos('fc_test0.mp4'))

### Some considerations:

Temperature at 0.4. 
During the training: with the first network the agent does not explore very well the map and often turns around the initial position. Conersely, with the convolutional network the agent better explores the map but still tends to rivisit many times some cells. in In both cases, during the testing the agent starts oscillating from the beginning.

Temperature at 0.8. 
During the training, in both cases the agent better explores the map since it can follow a path made by only red cases. During the testing, in both cases the agent explores just few cases before starting oscillating.

Temperature at 0.2. 
During the training procedure we observe the agent's behaviour is very similar to the case with temperature=0.4. During the testing: with the fully connected NN the agent starts oscillating since the very beginning, with the convolutional network the agent explores just few cases before starting oscillating.

# A4) Incorporating the dynamical (decreasing) $\epsilon$-greedy exploration and introducing a malus bonus for already visited status

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [ ]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    eps = 1
    decay_coeff = 0.99 
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        eps = eps*decay_coeff 
        agent.set_epsilon(eps) 
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            #agent.set_epsilon(agent.epsilon*decay_coeff)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over, reward_1 = env.act(action)

            # Update the counters
            if reward_1 > 0:
                win = round(win + reward_1,2)
                
            if reward_1 < 0:
                lose = round(lose - reward_1,2)

            # Apply the reinforcement strategy
            loss = round(agent.reinforce(prev_state, state, action, reward, game_over), 2)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += round(win-lose,2)
        

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board<0,2] = 256
        b[self.x,self.y,:]= 256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True): #train
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train: 
            reward = -self.malus_position[self.x, self.y]        
        self.malus_position[self.x, self.y] += 0.5 
        # 0.5 is the value that optimizes the average score. 
        
        reward += self.board[self.x, self.y]
        
        reward_1 = self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
          
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over, reward_1

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [ ]:
# Training
T = 50
epochs_train = 100 
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore0.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 4, 4, 32)          416       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          8256      
_________________________________________________________________
flatten_7 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 1024)              590848    
_________________________________________________________________
activation_5 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 4100      
_________________________________________________________________
activation_6 (Activation)    (None, 4)                 0         
Total para

In [ ]:
# Evaluation
epochs_test = 50
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 10.0/0. Average score (10.0)
Win/lose count 10.0/0. Average score (10.0)
Win/lose count 10.5/0. Average score (10.166666666666666)
Win/lose count 14.0/0. Average score (11.125)
Win/lose count 13.5/0. Average score (11.6)
Win/lose count 11.0/0. Average score (11.5)
Win/lose count 15.0/0. Average score (12.0)
Win/lose count 10.0/0. Average score (11.75)
Win/lose count 10.5/1.0. Average score (11.5)
Win/lose count 14.0/0. Average score (11.75)
Win/lose count 6.5/0. Average score (11.272727272727273)
Win/lose count 7.5/1.0. Average score (10.875)
Win/lose count 12.5/0. Average score (11.0)
Win/lose count 15.0/0. Average score (11.285714285714286)
Win/lose count 11.0/0. Average score (11.266666666666667)
Win/lose count 9.5/0. Average score (11.15625)
Win/lose count 10.5/0. Average score (11.117647058823529)
Win/lose count 10.5/1.0. Average score (11.027777777777779)
Win/lose count 16.0/0. Average score (11.289473684210526)
Win/lose count 10.0/1.0. Average score (11.175)
Win/l